In [ ]:
"""
Real-Time Earthquake Detection Using Live Infrasound PSD Data and CNN Model
---------------------------------------------------------------------------

This script performs continuous, real-time earthquake event detection by querying
live infrasound data from a Paros sensor, extracting PSD features, and running
inference through a trained 2D CNN model.

Workflow:
---------
1. Loads normalization statistics (mean and std) computed from training data.
2. Loads a pre-trained EarthquakeCNN2d model for earthquake classification.
3. Sets up CSV logging to record detected earthquake events along with timestamps.
4. Enters an infinite loop where every 60 seconds it:
   - Queries the most recent 60-second PSD feature vector using live_stream_query_for_model.
   - Preprocesses and normalizes the PSD vector.
   - Runs the CNN model to predict earthquake vs background probabilities.
   - Prints prediction results to the console.
   - Logs detected earthquake events (class 1) to a CSV file with timestamps and probabilities.
5. Handles graceful termination on user interrupt (Ctrl+C).

Input:
------
- Live PSD vectors computed from raw waveform data streamed via InfluxDB.

Output:
-------
- Console output of prediction results.
- Appended earthquake event records in "LoggedData/earthquake_predictions_log.csv".

Intended Use:
-------------
- Real-time monitoring system for detecting and logging earthquake infrasound events
  using machine learning on PSD features.

Author: Ethan Gelfand
Date: 08/12/2025
"""

import time
import numpy as np
import torch
import csv
import os
from datetime import datetime, timedelta, UTC
from cnn_model import EarthquakeCNN2d
from DataQueryUtils import live_stream_query_for_model

# Load normalization stats
mean = np.load("../DataCollection_Preprocessing/Exported_Paros_Data/mean.npy")
std = np.load("../DataCollection_Preprocessing/Exported_Paros_Data/std.npy")

# Load trained model
model = EarthquakeCNN2d(input_shape=(11,52))
model.load_state_dict(torch.load("../ModelTraining/fold_outputs/fold_5/CNNmodel.pth", map_location="cpu"))
model.eval()
model.to('cpu')

# CSV logging setup (create file and header if it doesn't exist)
dir = "LoggedData"
os.makedirs(dir, exist_ok=True)
log_path = "LoggedData/earthquake_predictions_log.csv"
with open(log_path, mode='a', newline='') as f:
    writer = csv.writer(f)
    f.seek(0, 2)  # Move to end of file
    if f.tell() == 0:
        writer.writerow(["timestamp", "window_start", "window_end", "predicted_class", "prob_earthquake", "prob_background"])

try:
    while True:
        query_time = datetime.now(UTC)
        window_end = query_time
        window_start = window_end - timedelta(seconds=60)

        psd_vector = live_stream_query_for_model(mean=mean, std=std)
        if psd_vector is not None:
            input_tensor = torch.tensor(psd_vector, dtype=torch.float32).unsqueeze(0)
            with torch.no_grad():
                output = model(input_tensor)
                probs = torch.softmax(output, dim=1).numpy()[0]
                pred = np.argmax(probs)

                print(f"Predicted class: {pred} | Probabilities: {probs}")

                # Log only if predicted class is earthquake (class 1)
                if pred == 1:
                    with open(log_path, mode='a', newline='') as f:
                        writer = csv.writer(f)
                        writer.writerow([
                            query_time.isoformat(timespec='seconds'),
                            window_start.isoformat(timespec='seconds'),
                            window_end.isoformat(timespec='seconds'),
                            int(pred),
                            round(float(probs[1]), 5),
                            round(float(probs[0]), 5)
                        ])
        else:
            print("No PSD vector returned from live_stream_query_for_model.")

        time.sleep(60)

except KeyboardInterrupt:
    print("Program terminated by user. Exiting gracefully.")


Querying data from 2025-08-12T22:09:57 to 2025-08-12T22:10:57
Error during stream: '_measurement'
No PSD vector returned from live_stream_query_for_model.
Program terminated by user. Exiting gracefully.
